In [ ]:
import wandb
from ultralytics import YOLO
import os
import torch
from roboflow import Roboflow

In [ ]:
rf = Roboflow(api_key=os.getenv("ROBOFLOW_API_KEY"))
workspace = os.getenv("ROBOFLOW_WORKSPACE", "your-workspace")
project_name = os.getenv("ROBOFLOW_PROJECT", "your-project")
version_num = int(os.getenv("ROBOFLOW_VERSION", "1"))
project = rf.workspace(workspace).project(project_name)
version = project.version(version_num)
dataset = version.download("yolov11")

In [ ]:
# Initialize wandb
wandb.init(
    project="Pose-augmented weapon detection using machine learning",
    name="yolo11s with 14k dataset",
    config={
        "epochs": 100,
        "batch_size": 16,
        "image_size": 640,
        "model": "yolov11s",
        "dataset": "14k dataset"
    }
)

In [ ]:
# Load a model
model = YOLO("src/yolo11s.pt")  # Load the default YOLOv11s small model

device = 'mps' if torch.mps.is_available() else 'cpu'  # Use mps if available
print(f"Using device: {device}")

In [ ]:
# Train the model
train_results = model.train(
    data='../data.yaml',  # path to your data.yaml
    epochs=100,  # number of training epochs
    imgsz=640,  # training image size
    device=device,  # device to run on
    batch=8,  # batch size
    save=True,  # save checkpoints
    project="src/runs/train",  # save results to project/name
    name="train",
    exist_ok=True,
    plots=True  # save plots
)

In [ ]:
# Log validation metrics to wandb
metrics = model.val()
wandb.log({
    "val/mAP50": metrics.box.map50,
    "val/mAP50-95": metrics.box.map,
    "val/precision": metrics.box.mp,
    "val/recall": metrics.box.mr
})

# Export the model and log it to wandb
path = model.export(format="onnx")
wandb.save(path)

# Finish wandb run
wandb.finish()